第10章 python进阶话题
---------------------------------
### 函数和命名空间
- 文件名作为命名空间，利用 import xxx 可以导入某个文件内的全部函数及其全局变量

### 闭包及其应用

- 闭包是指python语言中将组成函数的语句和这些语句的执行环境打包到一起所得到的对象，当使用嵌套函数时，闭包将捕获内部函数执行所需的整个环境。

In [2]:
#演示一个嵌套函数的例子
x = 14


def foo():
    x = 3

    def bar():
        print('x is %d' % x)

    bar()


foo()

x is 3


In [3]:
#闭包与延迟求值
def delay_fun(x, y):  #定义一个可以延迟求值的函数
    def calculator():  #内部嵌套的函数
        return x + y  #直接返回要求的值

    return calculator  #返回内部嵌套的函数对象


print("返回一个求和的函数，并不求和。")
msum = delay_fun(3, 4)  #调用外层函数,并不计算，实际返回一个函数对象
print()
print('调用并求和:')
print(msum())  #实际求值的调用

返回一个求和的函数，并不求和。

调用并求和:
7


In [4]:
#闭包的应用除了在装饰器和延迟求值外，还可以利用其特性来定义不同的泛型函数
def line(a, b):
    def aline(x):
        return a * x + b

    return aline


line23 = line(2, 3)
line50 = line(5, 0)
print(line23(4))  #输出2*4+3=11
print(line50(2))  #输出2*5+0=10

11
10


### 上下文管理器
- 上下文管理器是指实现上下文管理协议方法的对象，它主要用于安全地释放资源(如打开的文件、数据库连接或网络连接，对对象的锁定等)
- 对于上下文管理的对象可以使用with语句来使用它，在with语句中可以使用上下文管理器管理或提供资源，当退出with语句时，由上下文管理器来负责安全地释放资源
- 上下文管理器的协议方法有以下两个：
```python
__enter__(self)
__exit__(self,type,value,tb)
```
- 使用上下文管理器的with语句的形式为：
```python
with context as var:
    pass
```

In [7]:
class FileMgr: #实现上下文管理协议方法的类
    
    def __init__(self,filename): #定义构造函数
        self.filename = filename
        self.f = None
    def __enter__(self): # 定义协议方法
        print("before entering")
        self.f = open(self.filename, encoding = 'utf-8')
        return self.f
    def __exit__(self,t,v,tb):
        if self.f:
            self.f.close()
            print("already closed")
            
with FileMgr('./src//Chapter10fpa.txt') as f:
    for line in f.readlines():
        print(line)
    print("still in side of the context")

before entering
fpa-1

fpa-2

fpa-3

fpa-4
still in side of the context
already closed


In [8]:
# open函数也可以使用with管理
with open("./src//Chapter10fpa.txt") as f:
    for line in f:
        print(line)

fpa-1

fpa-2

fpa-3

fpa-4


- 此外，在python标准库中还有一个关于上下文管理器的模块contextlib, 其中还有一个可以将生成器转变为上下文管理器的修饰器，contextmanager

In [ ]:
import contextlib

#实现上下文管理器
@contextlib.contextmanager
def my_mgr(s, e):
    print(s) # 进入时调用
    yield s + ' yield ' + e # 产生返回值
    print(e) #退出时调用


with my_mgr('start', 'end') as val:
    result = val
    print(result)
    print("still in side of the context")

>代码的执行顺序是:  
1. with语句首先执行yield之前的语句，因此打印出start.  
2. yield 调用，result接收yield提供的返回值，因此打印 "start yield end".  
3. 执行with语句内部的所有语句，因此打印出“still in side of the context”.
4. 最后打印 “end”.

因此，@contextmanager让我们通过编写generator来简化上下文管理。

----------------------

### 用字符串操作对象属性
- 在python中内建函数中有两个函数: hasattr()和setattr()

 - hasattr(object,name) #测试某个对象是否有某个属性
 
 - setattr(object,name,value) # 设置某个对象属性值


In [ ]:
class DemoClass:
    class_val = 3

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        self.info()

    def info(self):
        print('类属性class_val:', DemoClass.class_val)  # 打印类属性值
        print("实例属性x:", self.x)
        print("实例属性y:", self.y)


dc = DemoClass(100, 200)  #实例化类
if hasattr(DemoClass, 'class_val'):
    setattr(DemoClass, 'class_val', 1000)  # 设置类属性的值
if hasattr(dc, 'x'):
    setattr(dc, 'x', 'xxxxxxxxx')
if hasattr(dc, 'y'):
    setattr(dc, 'y', 'yyyyyyyyy')

dc.info()
setattr(dc, 'z', 'zzzzzzzz')
print("添加的属性z", dc.z)

### 用字典构造分支程序

In [ ]:
import random


## 一下定义三个分支函数
def path_a():
    print('路径分支A')


def path_b():
    print("路径分支B")


def path_c():
    print("路径分支C")


path_dict = {}
path_dict['a'] = path_a
path_dict['b'] = path_b
path_dict['c'] = path_c
paths = 'abc'
for i in range(4):
    path = random.choice(paths)  #从所有的路径中随机选择一个路径
    print('选择的路径为:', path)
    path_dict[path]()  #进入分支程序

### 重载类的特殊方法
在python中，类中有一些以两条下划线开始和结束的方法，称之为类的专有方法。  
``` 
__init__ # 构造函数,生成对象时调用
__del__  #析构函数,释放对象时调用
__add__
__mul__
__cmp__
__repr__
__setitem__
__getitem__
__len__
__call__
```

In [ ]:
#自定义类来实现类的特别的运算方式
class Book:
    def __init__(self, name="Python从入门到精通"):
        self.name = name
    def __add__(self,obj):
        return self.name+" add "+obj.name
    
    def __len__(self):
        return len(self.name)
    
booka = Book()
bookb = Book('Java 从入门到精通')
print("len(booka):",len(booka))
print("len(bookb):",len(bookb))
print(booka+bookb)
    

### 鸭子类型
在python面向对象的编程中，没有谈及多态，也不用定义接口。在python语言中调用时是不检查参数类型的，如果被调用的方法不存在，则会引发错误。


In [ ]:
class Duck:
    def __init__(self, name='duck'):
        self.name = name

    def quack(self):
        print("嘎嘎嘎")


class Cat:
    def __init__(self, name="cat"):
        self.name = name

    def quack(self):
        print("喵喵喵")


class Tree:
    def __init__(self, name="tree"):
        self.name = name


def duck_demo(obj):  #定义了一个函数,接收任何obj对象，如果其有quack方法，则执行，否则会报错
    obj.quack()


duck = Duck()
cat = Cat()
tree = Tree()
duck_demo(duck)
duck_demo(cat)
duck_demo(tree)# 在这里会产生一个错误，因为tree类没有quack方法